In [ ]:
import os
import re
import shutil
import string


from tensorflow.keras import layers
from tensorflow.keras import losses

import tensorflow as tf 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, AveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, Activation, Convolution2D
from keras import backend as K
import matplotlib.pyplot as plt
import cv2
from keras.layers import Activation, Convolution2D, Dropout, Conv2D
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras import layers
from keras.regularizers import l2

# Load the datasets
data = pd.read_csv('fer2013 (11).csv', delimiter=',')#fer2013 (11)TrainLabels1

# Taking Training and PublicTest data for training and PrivateTest data for testing
data_train = data[:32298]
data_test = data[32298:]
# data_train.head()

y_train = data_train['emotion'].values
y_test = data_test['emotion'].values
print(y_train.shape)
print(y_test.shape)

# Converting string of pixel data to an array
x_train = np.zeros((y_train.shape[0], 48*48))
for i in range(y_train.shape[0]):
    x_train[i] = np.fromstring(data_train['pixels'][i], dtype=int, sep=' ')

x_test = np.zeros((y_test.shape[0], 48*48))
for i in range(y_test.shape[0]):
    x_test[i] = np.fromstring(data_test['pixels'][32298+i], dtype=int, sep=' ')

print(x_train.shape)
print(x_test.shape)

# Generate reversed images for every data assuming emotion are symetric
img_rows, img_cols = 48, 48
num_classes = 3  #7

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

x_train_rev = np.flip(x_train, 2)
x_test_rev = np.flip(x_test, 2)

plt.figure(1)
plt.imshow(x_train[0].reshape((48,48)))

plt.figure(2)
plt.imshow(x_train_rev[0].reshape((48,48)))

# Some preprocessing
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train_rev = x_train_rev.astype('float32')
x_test_rev = x_test_rev.astype('float32')
x_train /= 255
x_test /= 255
x_train_rev /= 255
x_test_rev /= 255
print('x_train shape:', x_train.shape)

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)

# define the model
def cnn_model():
    model = Sequential()

    model.add(BatchNormalization(input_shape=input_shape))

    model.add(Conv2D(32, kernel_size=(3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Conv2D(64, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(256, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(512, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])

    model.summary()
    return model
# model.summary()
def cnn_modell():
    model = Sequential()
    model.add(Convolution2D(filters=16, kernel_size=(7, 7), padding='same',
                                name='image_array', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=16, kernel_size=(7, 7), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(.5))

    model.add(Convolution2D(filters=32, kernel_size=(5, 5), padding='same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=32, kernel_size=(5, 5), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(.5))

    model.add(Convolution2D(filters=64, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=64, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(.5))

    model.add(Convolution2D(filters=128, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=128, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(.5))

    model.add(Convolution2D(filters=256, kernel_size=(3, 3), padding='same'))



    model.add(BatchNormalization())
    model.add(Convolution2D(filters=num_classes, kernel_size=(3, 3), padding='same'))
    model.add(GlobalAveragePooling2D())
    model.add(Activation('softmax',name='predictions'))

    model.summary()
    return model
# function to plot graph
def plotGraph(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'])
    plt.show()

    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'])
    plt.show()

# We will train two models one on normal images another on reversed images and finally a NN on predicted values from these models
import os 
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm

batch_size = 64
epochs = 20 #25
model = []

print("=======| Model 1 |=========")
modelc = cnn_model()
history = modelc.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split = 0.2)
model.append(modelc)
plotGraph(history)

print("=======| Model 2 |=========")
modelb = cnn_modell()
history = modelc.fit(x_train_rev, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split = 0.1)
model.append(modelb)
plotGraph(history)

# p_tr >> prediction on training data
# p_te >> prediction on test data

p_tr = []
p_te = []

for i, m in enumerate(model):
    if i ==0:
        p = m.predict(x_train)
        pt = m.predict(x_test)
    else:
        p = m.predict(x_train_rev)
        pt = m.predict(x_test_rev)
    p_tr.append(p)
    p_te.append(pt)
    m.save('saved_modell/cnn'+str(i)+'.h5')

print(len(model))

p_train = np.zeros((y_train.shape[0],num_classes*len(model)))
p_test = np.zeros((y_test.shape[0],num_classes*len(model)))
for i, p in enumerate(p_tr):
    print(i)
    p_train[:,num_classes*i:num_classes*(i+1)] = p

for i, p in enumerate(p_te):
    p_test[:,num_classes*i:num_classes*(i+1)] = p

print(p_train.shape, p_test.shape)

# Trains an Conventional Neural Network on previously predicted values by the two models

batch_size = 64 #32
num_classes = 3
epochs = 3

modele = Sequential()
modele.add(Dense(128, activation='relu', input_shape=(num_classes*len(model),)))
modele.add(Dense(num_classes, activation='softmax'))

modele.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()

history = modele.fit(p_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(p_test, y_test))

score = modele.evaluate(p_test, y_test, verbose=0)
modele.save('saved_modell/ensemble.h5')

print('NN Based Ensembled Model')
print('Test loss:', score[0])
print('Test accuracy:', score[1])


(32298,)
(3589,)
(32298, 2304)
(3589, 2304)
x_train shape: (32298, 48, 48, 1)
y_train shape: (32298, 3)
Segmentation Models: using `tf.keras` framework.
=======| Model 1 |=========
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 48, 48, 1)         4         
_________________________________________________________________
conv2d (Conv2D)              (None, 46, 46, 32)        320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 46, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 46, 46, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        18496     
_________________________________________________________________
batch_n

In [ ]:

import os
import re
import shutil
import string


from tensorflow.keras import layers
from tensorflow.keras import losses

import tensorflow as tf 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras import backend as K
import matplotlib.pyplot as plt
import cv2

# Load the datasets
data = pd.read_csv('TrainLabels1.csv', delimiter=',')#fer2013 (11)TrainLabels1

# Taking Training and PublicTest data for training and PrivateTest data for testing
data_train = data[:4000]
data_test = data[4000:]
# data_train.head()

y_train = data_train['emotion'].values
y_test = data_test['emotion'].values
print(y_train.shape)
print(y_test.shape)

# Converting string of pixel data to an array
x_train = np.zeros((y_train.shape[0], 48*48))
for i in range(y_train.shape[0]):
    x_train[i] = np.fromstring(data_train['ClipID'][i], dtype=int, sep=' ')

x_test = np.zeros((y_test.shape[0], 48*48))
for i in range(y_test.shape[0]):
    x_test[i] = np.fromstring(data_test['ClipID'][4000+i], dtype=int, sep=' ')

print(x_train.shape)
print(x_test.shape)

# Generate reversed images for every data assuming emotion are symetric
img_rows, img_cols = 48, 48
num_classes = 4  #7

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

x_train_rev = np.flip(x_train, 2)
x_test_rev = np.flip(x_test, 2)

plt.figure(1)
plt.imshow(x_train[0].reshape((48,48)))

plt.figure(2)
plt.imshow(x_train_rev[0].reshape((48,48)))

# Some preprocessing
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train_rev = x_train_rev.astype('float32')
x_test_rev = x_test_rev.astype('float32')
x_train /= 255
x_test /= 255
x_train_rev /= 255
x_test_rev /= 255
print('x_train shape:', x_train.shape)

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)

# define the model
def cnn_model():
    model = Sequential()

    model.add(BatchNormalization(input_shape=input_shape))

    model.add(Conv2D(32, kernel_size=(3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Conv2D(64, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(256, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    # model.add(Conv2D(32, (3, 3)))
    # model.add(BatchNormalization())
    # model.add(Activation('relu'))
    # model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])

    return model

# function to plot graph
def plotGraph(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'])
    plt.show()

    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'])
    plt.show()

# We will train two models one on normal images another on reversed images and finally a NN on predicted values from these models
import os 
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm

batch_size = 64
epochs = 5
model = []

print("=======| Model 1 |=========")
modelc = cnn_model()
history = modelc.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split = 0.2)
model.append(modelc)
plotGraph(history)

print("=======| Model 2 |=========")
modelc = cnn_model()
history = modelc.fit(x_train_rev, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split = 0.1)
model.append(modelc)
plotGraph(history)

# p_tr >> prediction on training data
# p_te >> prediction on test data

p_tr = []
p_te = []

for i, m in enumerate(model):
    if i ==0:
        p = m.predict(x_train)
        pt = m.predict(x_test)
    else:
        p = m.predict(x_train_rev)
        pt = m.predict(x_test_rev)
    p_tr.append(p)
    p_te.append(pt)
    m.save('saved_modell/cnn'+str(i)+'.h5')

print(len(model))

p_train = np.zeros((y_train.shape[0],num_classes*len(model)))
p_test = np.zeros((y_test.shape[0],num_classes*len(model)))
for i, p in enumerate(p_tr):
    print(i)
    p_train[:,num_classes*i:num_classes*(i+1)] = p

for i, p in enumerate(p_te):
    p_test[:,num_classes*i:num_classes*(i+1)] = p

print(p_train.shape, p_test.shape)

# Trains an Conventional Neural Network on previously predicted values by the two models

batch_size = 64 #32
num_classes = 4
epochs = 10

modele = Sequential()
modele.add(Dense(128, activation='relu', input_shape=(num_classes*len(model),)))
modele.add(Dense(num_classes, activation='softmax'))

modele.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

history = modele.fit(p_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(p_test, y_test))

score = modele.evaluate(p_test, y_test, verbose=0)
modele.save('saved_modell/ensemble.h5')

print('NN Based Ensembled Model')
print('Test loss:', score[0])
print('Test accuracy:', score[1])


(4000,)
(1358,)
(4000, 2304)
(1358, 2304)


<ipython-input-1-2c7a07a3dca4>:43: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  x_train[i] = np.fromstring(data_train['ClipID'][i], dtype=int, sep=' ')
<ipython-input-1-2c7a07a3dca4>:47: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  x_test[i] = np.fromstring(data_test['ClipID'][4000+i], dtype=int, sep=' ')


x_train shape: (4000, 48, 48, 1)
y_train shape: (4000, 4)
Segmentation Models: using `tf.keras` framework.
=======| Model 1 |=========
Epoch 1/5
50/50 [==============================] - 33s 618ms/step - loss: 0.9569 - accuracy: 0.6778 - val_loss: 0.8246 - val_accuracy: 0.7462
Epoch 2/5
26/50 [==============>...............] - ETA: 13s - loss: 0.8018 - accuracy: 0.7542